# Chapter 16: Functor

In [1]:
:info Functor

class Functor (f :: * -> *) where
  fmap :: (a -> b) -> f a -> f b
  (<$) :: a -> f b -> f a
  {-# MINIMAL fmap #-}
  	-- Defined in ‘GHC.Base’
instance Functor (Either a) -- Defined in ‘Data.Either’
instance Functor [] -- Defined in ‘GHC.Base’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Functor IO -- Defined in ‘GHC.Base’
instance Functor ((->) r) -- Defined in ‘GHC.Base’
instance Functor ((,) a) -- Defined in ‘GHC.Base’

In [2]:
data Bool = False | True
-- no functor because it must have kind * -> *

In [3]:
data BoolAndSomethingElse a = False' a | True' a

instance Functor BoolAndSomethingElse where
    fmap f (False' a) = False' (f a)
    fmap f (True' a) = True' (f a)

In [4]:
data BoolAndMaybeSomethingElse a = Falsish | Truish a

instance Functor BoolAndMaybeSomethingElse where
    fmap f Falsish = Falsish
    fmap f (Truish a) = Truish (f a)

In [5]:
newtype Mu f = InF { outF :: f (Mu f) }

-- no functor because fmap needs a function which transforms Functor argument
-- but here Functor argument (f) is a type constructor
-- we cannot have a function that accepts type constructor as an argument
-- and returns another type constructor

In [6]:
import GHC.Arr

data D = D (Array Word Word) Int Int

-- no functor because D is a concrete type

In [7]:
data Sum b a
    = First a
    | Second b
    
instance Functor (Sum e) where
    fmap f (First a) = First (f a)
    fmap f (Second b) = Second b

In [8]:
data Company a c b
    = DeepBlue a c
    | Something b
    
instance Functor (Company e e') where
    fmap f (Something b) = Something (f b)
    fmap _ (DeepBlue a c) = DeepBlue a c

In [9]:
data More b a
    = L a b a
    | R b a b
    deriving (Eq, Show)
    
instance Functor (More x) where
    fmap f (L a b a') = L (f a) b (f a')
    fmap f (R b a b') = R b (f a) b'
    
fmap (+1) (L 1 2 3)
fmap (+1) (R 1 2 3)

L 2 2 4

R 1 3 3

In [10]:
data Quant a b
    = Finance
    | Desk a
    | Bloor b
    
instance Functor (Quant a) where
    fmap _ (Desk a) = Desk a
    fmap f (Bloor b) = Bloor (f b)

In [11]:
newtype K a b = K a

instance Functor (K a) where
    fmap f (K a) = K a

In [12]:
{-# LANGUAGE FlexibleInstances #-}

newtype Flip f a b = Flip (f b a) deriving (Eq, Show)

instance Functor (Flip K a) where
    fmap f (Flip (K b)) = Flip (K $ f b)

In [13]:
newtype EvilGoateeConst a b = GoatyConst b

instance Functor (EvilGoateeConst a) where
    fmap f (GoatyConst b) = GoatyConst $ f b

In [14]:
newtype LiftItOut f a = LiftItOut (f a)

instance Functor f => Functor (LiftItOut f) where
    fmap a2b (LiftItOut fa) = LiftItOut (a2b <$> fa)

In [15]:
data Parappa f g a = DaWrappa (f a) (g a)

instance (Functor f, Functor g) => Functor (Parappa f g) where
    fmap a2b (DaWrappa fa ga) = DaWrappa (a2b <$> fa) (a2b <$> ga)

In [16]:
data IgnoreOne f g a b = IgnoringSomething (f a) (g b)

instance Functor g => Functor (IgnoreOne f g a) where
    fmap b2c (IgnoringSomething fa gb) = IgnoringSomething fa (b2c <$> gb)

In [17]:
data Notorious g o a t = Notorious (g o) (g a) (g t)

instance Functor g => Functor (Notorious g o a) where
    fmap t2t' (Notorious go ga gt) = Notorious go ga gt' where
        gt' = t2t' <$> gt

In [18]:
data List a
    = Nil
    | Cons a (List a)
    
instance Functor List where
    fmap _ Nil = Nil
    fmap f (Cons a as) = Cons (f a) (f <$> as)

In [19]:
data GoatLord a
    = NoGoat
    | OneGoat a
    | MoreGoats (GoatLord a) (GoatLord a) (GoatLord a)
    
instance Functor GoatLord where
    fmap _ NoGoat = NoGoat
    fmap f (OneGoat a) = OneGoat (f a)
    fmap f (MoreGoats a b c) = MoreGoats (f <$> a) (f <$> b) (f <$> c)    

In [20]:
data TalkToMe a
    = Halt
    | Print String a
    | Read (String -> a)
    
instance Functor TalkToMe where
    fmap _ Halt = Halt
    fmap f (Print str a) = Print str (f a)
    fmap f (Read r) = Read $ f . r